In [ ]:
# imports
import json
import os
import UnityPy
import re
from selenium import webdriver
from bs4 import BeautifulSoup
from random import randint
from time import sleep

In [ ]:
# Script to find missing assets

# Where the game is located up to the 0000 folder
GAME_PATH = "C:\\Program Files (x86)\\Steam\\steamapps\\common\\Yu-Gi-Oh!  Master Duel\\LocalData\\56b55645\\0000"

# Already collected data (same file used by the app)
data = json.load(open('../data.json', 'r+'))

ids = {"Card ID": {}, "Icons": {}, "adress": [], "field": []}

exist = []

# Build existing data list
for a in data['name']:
    exist.append(data['name'][a])

for a in data['icons']:
    for b in data['icons'][a]:
        exist.append(b)

exist.extend(data['adress'])
exist.extend(data['field'])

# Search every game folder [00 - ff]
for num in range(256):

    fPath = os.path.join(GAME_PATH, hex(num)[2:].zfill(2))

    for root, dirs, files in os.walk(fPath):
        for file_name in files:
            if file_name not in exist:

                file_path = os.path.join(root, file_name)

                env = UnityPy.load(file_path)

                for key in env.container.keys():
                    if "card/images/illust/common/" in key or "card/images/illust/tcg/" in key:
                        for obj in env.objects:
                            if obj.type.name == "Texture2D":
                                obj_data = obj.read()
                                ids["Card ID"][obj_data.name] = file_name
                                print(ids)
                    elif "images/profileicon/" in key:
                        # Profile icons
                        for obj in env.objects:
                            if obj.type.name == "Texture2D":
                                obj_data = obj.read()
                                ids["Icons"].setdefault(obj_data.name[11:18], []).append(file_name)
                                print(ids)
                    elif "assets/resourcesassetbundle/protector/common/" in key or "assets/resourcesassetbundle/protector/tcg/" in key:
                        # Sleeves
                        for obj in env.objects:
                            obj_data = obj.read()
                            if "ProtectorIcon" in obj_data.name and obj.type.name == "Texture2D":
                                ids["adress"].append(file_name)
                                print(ids)
                    elif re.search(r"mat_0([0-9][0-9])_near", key):
                        print(env.container.keys())
                        for obj in env.objects:
                            obj_data = obj.read()
                            if (hasattr(obj_data, 'name') and re.search(r"Mat_0([0-9][0-9])_near", obj_data.name)
                                    and obj.type.name == "Texture2D"):
                                # Fields
                                ids["field"].append(file_name)
                                print(ids)

                # for obj in env.objects:
                # 
                #     if obj.container is not None:
                # 
                #         data = obj.read()
                # 
                #         if "card/images/illust/common/" in obj.container or "card/images/illust/tcg/" in obj.container:
                #             # Cards
                #             ids["Card ID"][data.name] = file_name
                #             print(ids)
                #         elif "images/profileicon/" in obj.container:
                #             # Profile icons
                #             ids["Icons"].setdefault(data.name[11:18], []).append(file_name)
                #             print(ids)
                #         elif "assets/resourcesassetbundle/protector/common/" in obj.container or "assets/resourcesassetbundle/protector/tcg/" in obj.container:
                #             # Sleeves
                #             if "ProtectorIcon" in data.name and obj.type.name == "Texture2D":
                #                 ids["adress"].append(file_name)
                #                 print(ids)
                #         elif hasattr(data, 'name') and re.search(r"Mat_0([0-9][0-9])_near", data.name):
                #             # Fields
                #             ids["field"].append(file_name)
                #             print(ids)

    print(f"{hex(num)[2:].zfill(2)}/ff " + "■" * num + "□" * (256 - num), end='\r')

# jsonFile = open("sample.json")
# data = json.load(jsonFile)
# data.update(ids)
with open("sample.json", "w") as outfile:
    outfile.write(json.dumps(ids))

In [ ]:
# Script to scrape names from the KONAMI card database

if __name__ == '__main__':
    driver = webdriver.Edge()
    cards = {'name': {}}
    names = open("sample.json")
    ids = json.load(names)
    print(ids['Card ID'])

    for i in ids['Card ID']:
        driver.get(f"https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid={str(i)}&request_locale=pt")
        content = driver.page_source
        soup = BeautifulSoup(content)
        for a in soup.findAll('div', attrs={'id': 'cardname'}):
            name = a.find('h1')
            trueName = name.find('span')
            cards["name"][trueName.text] = ids['Card ID'][i]
            print(cards["name"][trueName.text])

        # Sleep to avoid possible IP banning
        sleep(randint(0, 2))

    with open("named.json", "w") as outfile:
        outfile.write(json.dumps(cards))

    driver.close()

In [ ]:
# Script to find cards without a standard ID (Usually tokens or alt arts)

if __name__ == '__main__':

    names = open("sample.json")
    ids = json.load(names)

    js = open("named.json")
    named = json.load(js)

    for e in ids['Card ID']:
        if ids['Card ID'][e] not in named["name"].values():
            print(f"\"{e}\": \"{ids['Card ID'][e]}\",")
